# <p style="background-color:#8B0000; font-family:newtimeroman;color:#FFF9ED; font-size:150%; text-align:center; border-radius: 15px 50px;"> CLTV Prediction with BG-NBD and Gamma-Gamma</p>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
    
### Business Problem

A shoe company wants to define a roadmap for sales and marketing activities.
In order for the company to make a medium to long term plan, it is necessary to estimate
the potential value that existing customers will provide to the company in the future.
> The dataset is private - that's the reason why you see it named as 'dataset'.


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
    
### Dataset Story
    
**The data set consists of information obtained from the past shopping behavior of customers
who made their last purchases as OmniChannel (both online and offline shoppers) in 2020 - 2021.**

**master_id :** Unique customer number

**order_channel :** Which channel of the shopping platform is used (Android, iOS, Desktop, Mobile, Offline)

**last_order_channel :** The channel where the last purchase was made

**first_order_date :** Date of the customer's first purchase

**last_order_date :** Date of the customer's last purchase

**last_order_date_online :** The date of the last purchase made by the customer on the online platform

**last_order_date_offline :** Date of the last purchase made by the customer on the offline platform

**order_num_total_ever_online :** Total number of purchases made by the customer on the online platform

**order_num_total_ever_offline :** Total number of purchases made by the customer offline

**customer_value_total_ever_offline :** Total price paid by the customer for offline purchases

**customer_value_total_ever_online :** Total price paid by the customer for online purchases

**interested_in_categories_12 :** List of categories the customer has shopped in the last 12 months



# <p style="background-color:#228B22; font-family:newtimeroman;color:#FFF9ED; font-size:100%; text-align:center; border-radius: 15px 50px;"> ⇣ Reading and Cleaning Data ⇣</p>


* Read the OmniChannel.csv data. Make a copy of the dataframe.

In [35]:
!pip install lifetimes 
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

df_ = pd.read_csv('/kaggle/input/shoe-company/shoe_data_20k.csv')
df = df_.copy()


* Define the outlier_thresholds and replace_with_thresholds functions to suppress outliers.

> Note: When calculating cltv, the frequency values must be integers, so round the lower and upper limits with round().


In [36]:
def outlier_thresholds(dataframe, variable):
    quartiles1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_rage = quartile3 - quartiles1
    up_limit = quartile3 + 1.5 * interquantile_rage
    low_limit = quartiles1 - 1.5 * interquantile_rage
    return round(low_limit), round(up_limit)

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

df.head()
df.isnull().sum()
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.1109,4.2256,1.0000,1.0000,2.0000,4.0000,200.0000
order_num_total_ever_offline,19945.0000,1.9139,2.0629,1.0000,1.0000,1.0000,2.0000,109.0000
customer_value_total_ever_offline,19945.0000,253.9226,301.5329,10.0000,99.9900,179.9800,319.9700,18119.1400
customer_value_total_ever_online,19945.0000,497.3217,832.6019,12.9900,149.9800,286.4600,578.4400,45220.1300


* Suppress any outliers of "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline", "customer_value_total_ever_online" variables.


In [37]:
for i in df.select_dtypes(include=['number']).columns:
    replace_with_thresholds(df, i)

df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.0920,3.8095,1.0000,1.0000,2.0000,4.0000,48.0000
order_num_total_ever_offline,19945.0000,1.8862,1.4347,1.0000,1.0000,1.0000,2.0000,16.0000
customer_value_total_ever_offline,19945.0000,251.9213,251.0237,10.0000,99.9900,179.9800,319.9700,3020.0000
customer_value_total_ever_online,19945.0000,489.7057,632.6098,12.9900,149.9800,286.4600,578.4400,7800.0000


* Omnichannel means that customers shop both online and offline.Create new variables for each customer's total number of purchases and spend.


In [38]:
df['total_order_num'] = df['order_num_total_ever_online'] + df['order_num_total_ever_offline']
df['total_order_value'] = df['customer_value_total_ever_offline'] + df['customer_value_total_ever_online']


* Examine the types of variables. Change the type of variables that express date to date.


In [39]:
for i in df.columns:
    if "date" in i:
        df[i] = pd.to_datetime(df[i])
df.info()        

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

# <p style="background-color:#228B22; font-family:newtimeroman;color:#FFF9ED; font-size:100%; text-align:center; border-radius: 15px 50px;"> ⇣ Setting up the CLTV Data Structure ⇣</p>

* Take 2 days after the date of the last purchase in the dataset as the analysis date.


In [40]:
today_date= df['last_order_date'].max() + pd.Timedelta(days = 2)

* Create a new cltv dataframe with customer_id, recency_cltv_weekly, T_weekly, frequency and monetary_cltv_avg.


In [41]:
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"]- df["first_order_date"]).dt.days) / 7
cltv_df["T_weekly"] = ((today_date - df["first_order_date"]).dt.days)/7
cltv_df["frequency"] = df["total_order_num"]
cltv_df["monetary_cltv_avg"] = df["total_order_value"] / df["total_order_num"]

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900


# <p style="background-color:#228B22; font-family:newtimeroman;color:#FFF9ED; font-size:100%; text-align:center; border-radius: 15px 50px;"> ⇣ Establishment of BG/NBD, Gamma-Gamma Models, calculation of 6-month CLTV ⇣</p>

* Setting up the BG/NBD model.

In [42]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

* Estimate the expected purchases from customers in 3 months and add it to the cltv dataframe as exp_sales_3_month.


In [43]:
cltv_df['exp_sales_3_month'] = bgf.predict(4*3,
                                           cltv_df['frequency'],
                                           cltv_df['recency_cltv_weekly'],
                                           cltv_df['T_weekly'])

* Estimate the expected purchases from customers in 6 months and add it to the cltv dataframe as exp_sales_6_month.


In [44]:
cltv_df['exp_sales_6_month'] = bgf.predict(4*6,
                                           cltv_df['frequency'],
                                           cltv_df['recency_cltv_weekly'],
                                           cltv_df['T_weekly'])

* Examine the top 10 purchasers in the 3rd and 6th month.


In [45]:
cltv_df.sort_values(by='exp_sales_3_month', ascending=False)[:10]
cltv_df.sort_values(by='exp_sales_6_month', ascending=False)[:10]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,52.0000,166.2246,4.6561,9.3123
15611,4a7e875e-e6ce-11ea-8f44-000d3a38a36f,39.7143,40.0000,29.0000,165.2976,3.3740,6.7479
8328,1902bf80-0035-11eb-8341-000d3a38a36f,28.8571,33.2857,25.0000,97.4396,3.1424,6.2848
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.7143,31.0000,228.5300,3.0838,6.1676
14373,f00ad516-c4f4-11ea-98f7-000d3a38a36f,38.0000,46.4286,27.0000,141.3548,3.0013,6.0026
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.1429,111.8571,43.0000,157.1126,2.9780,5.9561
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.1429,147.1429,49.0000,161.8467,2.8299,5.6598
6756,27310582-6362-11ea-a6dc-000d3a38a36f,62.7143,64.1429,29.0000,168.8810,2.7934,5.5869
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.7143,13.0000,17.0000,259.8653,2.7807,5.5614
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.5714,113.4286,40.0000,176.2000,2.7635,5.5270


* Fit the Gamma-Gamma model. Estimate the average value that customers will leave and add it to the cltv dataframe as exp_average_value.

In [46]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
cltv_df['exp_average_value'] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                       cltv_df['monetary_cltv_avg'])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,193.6327
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.6650
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,120.9676
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,67.3201
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,114.3251


* Calculate the CLTV for 6 months and add it to the dataframe as cltv.


In [47]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time= 6, # 6 month prediction
                                   freq= 'W', # The frequency of T
                                   discount_rate= 0.01)

cltv_df["cltv"] = cltv

* Observe the 20 people with the highest CLTV values.

In [48]:
cltv_df.sort_values(by='cltv',ascending=False)[:20]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,7.8571,4.0000,1401.8000,1.0944,2.1888,1449.0605,3327.7770
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.1429,11.0000,758.0855,1.9701,3.9402,767.3606,3172.3944
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.0000,7.0000,1106.4671,0.7222,1.4445,1127.6115,1708.9821
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.2857,74.5714,16.0000,501.8737,1.5653,3.1306,506.1667,1662.6135
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,52.0000,166.2246,4.6561,9.3123,166.7123,1628.8874
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.4286,34.4286,8.0000,601.2262,1.2655,2.5309,611.4926,1623.8127
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.2857,9.4286,2.0000,862.6900,0.7939,1.5878,923.6800,1538.8559
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.7143,13.0000,17.0000,259.8653,2.7807,5.5614,262.0729,1529.2280
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.7143,31.0000,228.5300,3.0838,6.1676,229.6069,1485.8192
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.8571,15.5714,3.0000,743.5867,0.8716,1.7431,778.0504,1422.9997


# <p style="background-color:#228B22; font-family:newtimeroman;color:#FFF9ED; font-size:100%; text-align:center; border-radius: 15px 50px;"> ⇣ Creation of Segments According to CLTV ⇣</p>

* Divide all your customers into 4 groups (segments) according to 6 months CLTV and add the group names to the dataset, assign them with the name cltv_segment.


In [49]:
cltv_df['cltv_segment'] = pd.qcut(cltv_df['cltv'], 4, labels= ["D", "C", "B", "A"])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,193.6327,395.7332,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.6650,199.4307,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,120.9676,170.2242,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,67.3201,98.9455,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,114.3251,95.0117,D


* Examine the recency, frequnecy and monetary averages of the segments.


In [50]:
cltv_df.groupby('cltv_segment').agg({'recency_cltv_weekly': 'mean',
                                     'frequency': 'mean',
                                     'monetary_cltv_avg': 'mean'})

,recency_cltv_weekly,frequency,monetary_cltv_avg
cltv_segment,,,
D,139.0002,3.7688,93.1516
C,92.6295,4.4047,125.7885
B,81.9884,5.0927,160.6366
A,67.4271,6.6466,228.8311


# <p style="background-color:#228B22; font-family:newtimeroman;color:#FFF9ED; font-size:100%; text-align:center; border-radius: 15px 50px;"> ⇣ BONUS: Functionalize the whole process ⇣</p>

In [51]:

def create_cltv_df(dataframe):

    # Data Preparation
    columns = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline","customer_value_total_ever_online"]
    for col in columns:
        replace_with_thresholds(dataframe, col)

    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    dataframe = dataframe[~(dataframe["customer_value_total"] == 0) | (dataframe["order_num_total"] == 0)]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

    # Setting up the CLTV data structure
    dataframe["last_order_date"].max()  # 2021-05-30
    analysis_date = dt.datetime(2021, 6, 1)
    cltv_df = pd.DataFrame()
    cltv_df["customer_id"] = dataframe["master_id"]
    cltv_df["recency_cltv_weekly"] = ((dataframe["last_order_date"] - dataframe["first_order_date"]).dt.days) / 7
    cltv_df["T_weekly"] = ((analysis_date - dataframe["first_order_date"]).dt.days) / 7
    cltv_df["frequency"] = dataframe["order_num_total"]
    cltv_df["monetary_cltv_avg"] = dataframe["customer_value_total"] / dataframe["order_num_total"]
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

    # Setting up the BG-NBD Model
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency_cltv_weekly'],
            cltv_df['T_weekly'])
    cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                               cltv_df['frequency'],
                                               cltv_df['recency_cltv_weekly'],
                                               cltv_df['T_weekly'])
    cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                               cltv_df['frequency'],
                                               cltv_df['recency_cltv_weekly'],
                                               cltv_df['T_weekly'])

    # Setting up the Gamma-Gamma Model
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
    cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                           cltv_df['monetary_cltv_avg'])

    # CLTV prediction
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'],
                                       cltv_df['monetary_cltv_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)
    cltv_df["cltv"] = cltv

    # CLTV segmentation
    cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_df

cltv_df = create_cltv_df(df)


cltv_df.head(10)


/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,193.6327,395.7332,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.6650,199.4307,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,120.9676,170.2242,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,67.3201,98.9455,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,114.3251,95.0117,D
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,120.8571,132.2857,3.0000,66.9533,0.3836,0.7672,71.3481,57.4299,D
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,32.5714,64.8571,4.0000,93.9825,0.6521,1.3041,98.1334,134.2784,C
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,12.7143,54.5714,2.0000,81.8150,0.5198,1.0396,89.5702,97.7006,D
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,58.4286,70.7143,5.0000,210.9380,0.7078,1.4155,217.3009,322.7347,A
9,1143f032-440d-11ea-8b43-000d3a38a36f,61.7143,96.0000,2.0000,82.9800,0.3947,0.7894,90.8146,75.2224,D
